In [67]:
!pip install azureml
!pip install azureml.core
!pip install azureml.widgets
!pip install azureml.pipeline

  Using cached azureml_core-1.28.0-py3-none-any.whl (2.2 MB)
ERROR: azureml-defaults 1.21.0 has requirement azureml-core~=1.21.0, but you'll have azureml-core 1.28.0 which is incompatible.
ERROR: azureml-defaults 1.21.0 has requirement azureml-dataset-runtime[fuse]~=1.21.0, but you'll have azureml-dataset-runtime 1.28.0 which is incompatible.
ERROR: azureml-defaults 1.21.0 has requirement flask==1.0.3, but you'll have flask 1.1.2 which is incompatible.


  Using cached azureml_widgets-1.28.0-py3-none-any.whl (14.1 MB)


  Using cached azureml_pipeline-1.28.0-py3-none-any.whl (3.7 kB)


In [68]:
import os
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset, Run
from azureml.widgets import RunDetails
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

In [69]:
ws = Workspace.from_config()

In [70]:
keyvault = ws.get_default_keyvault()

In [71]:
default_datastore = ws.get_default_datastore()

In [72]:
dataset_kaggle_cord19_name = 'kaggle-cord19'

# Get a dataset by name
dataset_kaggle_cord19 = Dataset.get_by_name(workspace=ws, name=dataset_kaggle_cord19_name)

### Create CPU Compute

In [73]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# CPU cluster name and sku
aml_cpu_compute_cluster_name = 'cpuclust-f32sv2'
vm_size = 'STANDARD_F32S_V2'

# Check cluster does not exist already
try:
    aml_cpu_compute = ComputeTarget(workspace=ws, name=aml_cpu_compute_cluster_name)
    print('Found existing cluster {}'.format(aml_cpu_compute_cluster_name))
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           max_nodes=8)
    aml_cpu_compute = ComputeTarget.create(ws, aml_cpu_compute_cluster_name, compute_config)

aml_cpu_compute.wait_for_completion(show_output=True)

Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Define Runtime Configuration

In [74]:
# https://docs.microsoft.com/en-us/azure/container-registry/container-registry-auth-service-principal

from azureml.core.runconfig import RunConfiguration
aml_run_config = RunConfiguration()

from azureml.core.environment import Environment
# Create the environment
rp_env = Environment(name='rp')
# Enable Docker and reference an image
rp_env.docker.enabled = True
rp_env.python.user_managed_dependencies=True

rp_env.docker.base_image_registry.address = "acieurfrcaassacr.azurecr.io"
rp_env.docker.base_image_registry.username = keyvault.get_secret(name='acieurfrcaassacr-admin-user')
rp_env.docker.base_image_registry.password = keyvault.get_secret(name='acieurfrcaassacr-admin-pwd')
rp_env.docker.base_image = "acieurfrcaassacr.azurecr.io/azureml-env-base-research-platform:latest"
                          
aml_run_config.environment = rp_env
rp_env.register(workspace=ws)

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "acieurfrcaassacr.azurecr.io/azureml-env-base-research-platform:latest",
        "baseImageRegistry": {
            "address": "acieurfrcaassacr.azurecr.io",
            "password": "AzureMlSecret=Env_796e80c7e5bf8a48cb603f229eeec578ec72443dbfe37710cc80de67228c6713_2#EnvironmentDefinition#ContainerRegistry#Password",
            "registryIdentity": null,
            "username": "acieurfrcaassacr"
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "rp",
    

## Specify script source folder(s)

In [75]:
dataprep_script_folder = '../'

print('Source directory for data preparation is {}.'.format(os.path.realpath(dataprep_script_folder)))

Source directory for data preparation is /media/shared/My Documents/My Developments/research-platform/code/dataprep/cord19.


### Step - Extract

In [76]:
dataset_size_percent = 100
step_dataprep_extract_out = PipelineData('extract_out', datastore=default_datastore)

step_dataprep_extract = PythonScriptStep(
    name='Extract',
    script_name='step_dataprep_extract.py', 
    arguments=['--input', dataset_kaggle_cord19.as_named_input('raw_data').as_mount(),
               '--dataset_size_percent', dataset_size_percent,
               '--pmc_only', 'n',
               '--output', step_dataprep_extract_out],
    outputs=[step_dataprep_extract_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Extract Created')

Step Extract Created


### Step - Language Detection

In [77]:
step_dataprep_lang_detect_out = PipelineData('lang_detect_out', datastore=default_datastore)

step_dataprep_lang_detect = PythonScriptStep(
    name='Language Detection',
    script_name='step_dataprep_lang_detect.py', 
    arguments=['--input', step_dataprep_extract_out,
               '--max_doc_length', 1000,
               '--output', step_dataprep_lang_detect_out],
    inputs=[step_dataprep_extract_out],
    outputs=[step_dataprep_lang_detect_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Language Detection Created')

Step Language Detection Created


### Step - Create Hash Index

In [78]:
step_dataprep_hash_index_out = PipelineData('hash_index_out', datastore=default_datastore)

step_dataprep_hash_index = PythonScriptStep(
    name='Hash Index',
    script_name='step_dataprep_hash_index.py', 
    arguments=['--input', step_dataprep_lang_detect_out,
               '--output', step_dataprep_hash_index_out],
    inputs=[step_dataprep_lang_detect_out],
    outputs=[step_dataprep_hash_index_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Hash Index Created')

Step Hash Index Created


### Step - Stopwords & Lemmatize

In [79]:
step_dataprep_stopwords_lemma_out = PipelineData('stopwords_lemma_out', datastore=default_datastore)

step_dataprep_stopwords_lemma = PythonScriptStep(
    name='Stopwords & Lemmatization',
    script_name='step_dataprep_stopwords_lemma.py', 
    arguments=['--input', step_dataprep_hash_index_out,
               '--output', step_dataprep_stopwords_lemma_out],
    inputs=[step_dataprep_hash_index_out],
    outputs=[step_dataprep_stopwords_lemma_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Stopwords Lemma Created')

Step Stopwords Lemma Created


### Step - Trials

In [80]:
step_dataprep_trials_out = PipelineData('trials_out', datastore=default_datastore)

step_dataprep_trials = PythonScriptStep(
    name='Trials',
    script_name='step_dataprep_trials.py', 
    arguments=['--input', step_dataprep_hash_index_out,
               '--output', step_dataprep_trials_out],
    inputs=[step_dataprep_hash_index_out],
    outputs=[step_dataprep_trials_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Trials created')

Step Trials created


### Step - Feature Engineering

In [81]:
step_dataprep_feature_engineering_out = PipelineData('feature_engineering_out', datastore=default_datastore)

step_dataprep_feature_engineering = PythonScriptStep(
    name='Feature Engineering',
    script_name='step_dataprep_feature_engineering.py', 
    arguments=['--input', step_dataprep_hash_index_out,
               '--output', step_dataprep_feature_engineering_out],
    inputs=[step_dataprep_hash_index_out],
    outputs=[step_dataprep_feature_engineering_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Feature Engineering created')

Step Feature Engineering created


### Step - Author Article Metrics

In [82]:
step_dataprep_author_article_metrics_out = PipelineData('author_article_metrics_out', datastore=default_datastore)
step_dataprep_author_info_out = PipelineData('author_info_out', datastore=default_datastore)
step_dataprep_authors_by_paper_out = PipelineData('authors_by_paper_out', datastore=default_datastore)

step_dataprep_author_article_metrics = PythonScriptStep(
    name='Author & Article Metrics',
    script_name='step_dataprep_author_article_metrics.py', 
    arguments=['--input', step_dataprep_feature_engineering_out,
               '--output_author_article_metrics', step_dataprep_author_article_metrics_out,
               '--output_author_info', step_dataprep_author_info_out,
               '--output_authors_by_paper', step_dataprep_authors_by_paper_out],
    inputs=[step_dataprep_feature_engineering_out],
    outputs=[step_dataprep_author_article_metrics_out, 
             step_dataprep_author_info_out,
             step_dataprep_authors_by_paper_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Author Article Metrics Created')

Step Author Article Metrics Created


### Step - Topic Modelling

In [83]:
step_dataprep_topic_modelling_topics_out = PipelineData('topic_modelling_topics_out', datastore=default_datastore)
step_dataprep_topic_modelling_document_topic_out = PipelineData('topic_modelling_document_topic_out', datastore=default_datastore)

step_dataprep_topic_modelling = PythonScriptStep(
    name='Topic Modelling',
    script_name='step_dataprep_topic_modelling.py', 
    arguments=['--input', step_dataprep_stopwords_lemma_out,
               '--n_topics', 5,
               '--n_top_words', 20,
               '--no_optimise',
               '--output_topics', step_dataprep_topic_modelling_topics_out,
               '--output_document_topic', step_dataprep_topic_modelling_document_topic_out],
    inputs=[step_dataprep_stopwords_lemma_out],
    outputs=[step_dataprep_topic_modelling_topics_out,
             step_dataprep_topic_modelling_document_topic_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Topic Modelling Created')

Step Topic Modelling Created


### Step - Database Update

In [84]:
step_database_update = PipelineData('database_update', datastore=default_datastore)

step_database_update = PythonScriptStep(
    name='Database Update',
    script_name='step_database_update.py',
    arguments=['--input_topic_modelling_topics', step_dataprep_topic_modelling_topics_out,
               '--input_topic_modelling_document_topic', step_dataprep_topic_modelling_document_topic_out,
               '--input_feature_engineering', step_dataprep_feature_engineering_out,
               '--input_author_info', step_dataprep_author_info_out,
               '--input_author_article_metrics', step_dataprep_author_article_metrics_out,
               '--input_authors_by_paper', step_dataprep_authors_by_paper_out,
               '--input_trials', step_dataprep_trials_out],
    inputs=[step_dataprep_topic_modelling_topics_out,
            step_dataprep_topic_modelling_document_topic_out,
            step_dataprep_feature_engineering_out,
            step_dataprep_author_info_out,
            step_dataprep_author_article_metrics_out,
            step_dataprep_authors_by_paper_out,
            step_dataprep_trials_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Database Update Created')

Step Database Update Created


### Create Step List

In [85]:
steps = [step_dataprep_extract, 
         step_dataprep_lang_detect, 
         step_dataprep_hash_index,
         step_dataprep_stopwords_lemma, 
         step_dataprep_author_article_metrics, 
         step_dataprep_feature_engineering, 
         step_dataprep_topic_modelling,
         step_dataprep_trials,
         step_database_update]
print("Step lists created")

Step lists created


In [86]:
aml_pipeline = Pipeline(workspace=ws, steps=steps)
print ('Pipeline is built')

Pipeline is built


In [87]:
aml_pipeline.validate()
print('Pipeline validation complete')

Step Extract is ready to be created [d5821fb8]
Step Language Detection is ready to be created [7cb9f2c2]
Step Hash Index is ready to be created [b4230dc0]
Step Stopwords & Lemmatization is ready to be created [71fe3ba0]
Step Author & Article Metrics is ready to be created [82ff8028]
Step Feature Engineering is ready to be created [29f7a772]
Step Topic Modelling is ready to be created [1ed32456]Step Trials is ready to be created [8d637e60]
Step Database Update is ready to be created [f151a911]

Pipeline validation complete


In [88]:
# Note regenerate output, decides whether to execute the module to regenerate output at running time.
# It's by default unselected, which means if the module has been executed with the same parameters previously, 
# the system will reuse the output from last run to reduce run time.

aml_pipeline_run = Experiment(ws, 'pipeline-cord19-dataprep').submit(aml_pipeline, regenerate_outputs=False)
print('Experiment pipeline cord19 dataprep is submitted for execution')

Created step Extract [d5821fb8][20f24e7d-8d3a-4fb7-8769-feabb9c3a739], (This step will run and generate new outputs)Created step Language Detection [7cb9f2c2][999a90c6-8fa3-4906-95e5-9755a108d0ef], (This step will run and generate new outputs)

Created step Hash Index [b4230dc0][45575a45-d2ab-4cb7-a1ec-cfe1df00a26a], (This step will run and generate new outputs)
Created step Stopwords & Lemmatization [71fe3ba0][e82af774-8215-494b-8782-31fd670f5c76], (This step will run and generate new outputs)
Created step Author & Article Metrics [82ff8028][e1e6bcb1-b170-4022-bc91-18bd80865e88], (This step will run and generate new outputs)
Created step Feature Engineering [29f7a772][5b8e9bab-f536-46fa-9430-0df4b00642d4], (This step will run and generate new outputs)
Created step Topic Modelling [1ed32456][1345e982-968f-42c4-82c4-1672d252a01e], (This step will run and generate new outputs)
Created step Trials [8d637e60][23a30985-68d0-42c4-9ac1-edd7d9b6a459], (This step will run and generate new outpu

In [89]:
RunDetails(aml_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [ ]:
step_runs = aml_pipeline_run.get_children()
for step_run in step_runs:
    status = step_run.get_status()
    print('Script:', step_run.name, 'status:', status)
    
    # Change this if you want to see details even if the Step has succeeded.
    if status == "Failed":
        joblog = step_run.get_job_log()
        print('job log:', joblog)